In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
#export
from exp.nb_00 import *
def get_data():
    path = datasets.download_data(MNIST_URL, ext ='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train),(x_valid, y_valid),_) = pickle.load(f, encoding = 'latin-1')
    return map(tensor, (x_train,y_train, x_valid, y_valid))

def normalize(x,m,s):
    return (x-m)/s

???


In [11]:
x_train,y_train,x_valid,y_valid = get_data()


In [12]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [13]:
x_train = normalize(x_train,train_mean,train_std)
x_valid = normalize(x_valid,train_mean,train_std)


In [14]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean, train_std

(tensor(-7.6999e-06), tensor(1.))

In [15]:
 #export
def test_near_zero(a,tol = 1e-3): assert a.abs()<tol, f"Near zero:{a}"

In [17]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std() - 1)

In [18]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

In [21]:
#hidden layers
nh = 50

In [23]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [25]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1/math.sqrt(m))

In [27]:
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [20]:
def lin(x,w,b):return x@w+b

In [28]:
t = lin(x_valid,w1,b1)

In [29]:
t.mean(),t.std()

(tensor(-0.0137), tensor(0.9263))

In [30]:
#Fixup Initialization

In [31]:
def relu(x):return x.clamp_min(0.)

In [32]:
t = relu(lin(x_valid,w1,b1))

In [53]:
t.mean(),t.std()

(tensor(0.5242), tensor(0.8237))

In [46]:
w1 = torch.randn(m,nh)*math.sqrt(2/m)


In [47]:
w1.mean(),w1.std()

(tensor(0.0004), tensor(0.0508))

In [48]:
t = relu(lin(x_valid,w1,b1))

In [49]:
t.mean(),t.std()

(tensor(0.4500), tensor(0.7584))

In [50]:
#export
from torch.nn import init

In [51]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1,mode='fan_out')#fan_in or fan_out divided by m or nh
t = relu(lin(x_valid,w1,b1))

In [52]:
t.mean(),t.std()

(tensor(0.5242), tensor(0.8237))

In [54]:
w1.shape

torch.Size([784, 50])

In [56]:
torch.nn.Linear(m,nh).weight.shape# also do a transpose

torch.Size([50, 784])

In [58]:
def relu(x):return x.clamp_min(0.) - 0.5

In [59]:
t = relu(lin(x_valid,w1,b1))

In [60]:
t.mean(),t.std()

(tensor(0.0242), tensor(0.8237))

In [71]:
def model(xb):
    l1 = lin(xb,w1,b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [63]:
%timeit -n 10 _ = model(x_valid)

5.14 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

In [65]:
model(x_valid).shape

torch.Size([10000, 1])

In [66]:
def mse(output, targ): return (output.squeeze()-targ).pow(2).mean()

In [67]:
y_train, y_valid = y_train.float(),y_valid.float()

In [72]:
preds = model(x_train)

In [73]:
preds.shape

torch.Size([50000, 1])

In [91]:
y_train[0]

tensor(5.)

In [82]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [83]:
def relu_grad(inp,out):
    inp.g = (inp>0).float()*out.g

In [84]:
def lin_grad(inp,out,w,b):
    inp.g = out.g@w.t()
    w.g = (inp.unsqueeze(-1)*out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [85]:
def forward_and_backward(inp, targ):
     # forward pass:\n",
        l1 = inp @ w1 + b1
        l2 = relu(l1)
        out = l2 @ w2 + b2
        # we don't actually need the loss in backward!\n",
        loss = mse(out, targ)
        # backward pass:\n",
        mse_grad(out, targ)
        lin_grad(l2, out, w2, b2)
        relu_grad(l1, l2)
        lin_grad(inp, l1, w1, b1)
    

In [92]:
forward_and_backward(x_train,y_train)

???
